## Домашнее задание 3 
**Выполнили Гайдук Юлия, Панкратова Анна МЖД172**

Задание: Взять коллекцию текстов (не менее 100Кб), построить базовые частотные списки, построить список ключевых слов используя алгоритм RAKE, сравнить полученные результаты.


Работа выполняется на материале произведения Е. И. Рериха ["Мир Огненный"](http://www.lib.ru/RERIH/Rerih_Mir1.txt) объемом 2 Мб

In [39]:
import re
import sys

import pymorphy2
from rake_nltk import Metric, Rake
from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer

import matplotlib.pyplot as plt
%matplotlib inline

sys.path.append(r'C:\Users\user\AppData\Local\Programs\Python\Python36-32\Lib\site-packages')

In [7]:
# скрипт из репо
def lemmatize_pm(fname, text):
    """
    fname: название файла, куда будет записан результат лемматизации
    text: текст для анализа
    """
    tokens = word_tokenize(text)
    morph = pymorphy2.MorphAnalyzer()

    with open(fname, 'w', encoding='utf-8') as outfile:
        for token in tokens:
            parsed = morph.parse(token)
            outfile.write(parsed[0].normal_form + ' ')

In [14]:
def prepare_text(text):
    # Убирает знаки препинания и символы. 
    text = re.sub(r'[^\w\s]','', text)
    text = re.sub(r'[0-9]+', '', text)
    text = text.replace('\n','')
    text = text.replace('\x14','')
    text = text.replace('\x15','')
    text = text.replace('pp','')
    return text

def normalize_text(text):
    # На вход подаётся подготовленный список слов. Возвращает список лемм.
    morph = pymorphy2.MorphAnalyzer()
    normed_text = []
    for word in text:
        normed_word = morph.parse(word)[0].normal_form
        normed_text.append(normed_word)
    return normed_text

In [12]:
with open("Rerih_Mir.txt") as f:
    text = f.read()
text = prepare_text(text)

In [13]:
lemmatize_pm('lemmas_pm.txt', text)

In [15]:
# загружаем список стоп-слов
with open("ru_stop_words.txt") as f:
    stop_words = f.read()
stop_words = stop_words.split("\n")
stop_words.pop(-1)

''

In [17]:
# очищаем текст от стоп-слов
with open("lemmas_pm.txt") as f:
    lemmas_pm = f.read().split()
processed_text = [word for word in lemmas_pm if word not in stop_words]

In [23]:
def get_Afreq(text):
    # На вход подаётся список элементов.
    # Возвращает словарь абсолютных частот, где ключами являются элементы, 
    # а значениями — количество появлений элемента в тексте.
    from collections import Counter
    c = Counter()
    for word in text:
        c.update({word:1})
    return dict(c)

In [41]:
from operator import itemgetter
Afreq_text = get_Afreq(processed_text)
Afreq_list_sorted = sorted(Afreq_text.items(), key=itemgetter(-1), reverse=True)
print('Частотный список')
for tpl in Afreq_list_sorted[:20]:
    print("Лемма '{}' встречается в тексте {} раз.".format(tpl[0], tpl[1]))

Частотный список
Лемма 'огненный' встречается в тексте 2168 раз.
Лемма 'мир' встречается в тексте 1705 раз.
Лемма 'нужно' встречается в тексте 1291 раз.
Лемма 'можно' встречается в тексте 1163 раз.
Лемма 'человек' встречается в тексте 1144 раз.
Лемма 'дух' встречается в тексте 980 раз.
Лемма 'такой' встречается в тексте 966 раз.
Лемма 'огонь' встречается в тексте 883 раз.
Лемма 'сердце' встречается в тексте 881 раз.
Лемма 'который' встречается в тексте 881 раз.
Лемма 'каждый' встречается в тексте 826 раз.
Лемма 'энергия' встречается в тексте 813 раз.
Лемма 'когда' встречается в тексте 809 раз.
Лемма 'тонкий' встречается в тексте 747 раз.
Лемма 'самый' встречается в тексте 710 раз.
Лемма 'сознание' встречается в тексте 691 раз.
Лемма 'высокий' встречается в тексте 664 раз.
Лемма 'свой' встречается в тексте 643 раз.
Лемма 'мысль' встречается в тексте 634 раз.
Лемма 'явление' встречается в тексте 614 раз.


Теперь применим алгоритм RAKE.

In [42]:
split_text = text.split()
norm_text = normalize_text(split_text)
prep_text = " ".join(norm_text)

In [43]:
# Лемматизированный текст, сортировка по частоте, длина 2
r = Rake(language="russian", stopwords=stop_words, max_length=2, ranking_metric=Metric.WORD_FREQUENCY)
r.extract_keywords_from_text(prep_text)
freq_2 = r.get_ranked_phrases() 
print("Наиболее часто встречающиеся словосочетания длины 2 по лемматизированному тексту:")
for item in freq_2[:10]:
    print(item)

Наиболее часто встречающиеся словосочетания длины 2 по лемматизированному тексту:
огненный дух
дух человек
только человек
огненный сознание
сердце огненный
огненный сердце
сознание человек
огонь дух
огненный мир
мир огненный


In [44]:
# Лемматизированный текст, сортировка по частоте, длина 3
r = Rake(language="russian", stopwords=stop_words, max_length=3, ranking_metric=Metric.WORD_FREQUENCY)
r.extract_keywords_from_text(prep_text)
freq_3 = r.get_ranked_phrases() # Сортировка по частоте, длина 3
print("Наиболее часто встречающиеся словосочетания длины 3 по лемматизированному тексту:")
for item in freq_3[:10]:
    print(item)

Наиболее часто встречающиеся словосочетания длины 3 по лемматизированному тексту:
мир огненный человек
огненный мир нужно
мир огненный нужно
мир огненный можно
мир огненный сердце
энергия огненный мир
огненный мир когда
мир огненный когда
земной мир огненный
мир огненный ибо


In [45]:
# Нелемматизированный текст, сортировка по частоте, длина 2
r = Rake(language="russian", stopwords=stop_words, max_length=2, ranking_metric=Metric.WORD_FREQUENCY)
r.extract_keywords_from_text(text)
freq_2_simple = r.get_ranked_phrases() # Сортировка по частоте, длина 2
print("Наиболее часто встречающиеся словосочетания длины 2 по нелемматизированному тексту:")
for item in freq_2_simple[:10]:
    print(item)

Наиболее часто встречающиеся словосочетания длины 2 по нелемматизированному тексту:
может сердце
будет нужно
только могут
жизни может
сознание может
нужно сердце
можно будет
огонь может
огненного нужно
только тонкого


In [46]:
# Нелемматизированный текст, сортировка по частоте, длина 2
r = Rake(language="russian", stopwords=stop_words, max_length=3, ranking_metric=Metric.WORD_FREQUENCY)
r.extract_keywords_from_text(text)
freq_3_simple = r.get_ranked_phrases() # Сортировка по частоте, длина 3
print("Наиболее часто встречающиеся словосочетания длины 3 по нелемматизированному тексту:")
for item in freq_3_simple[:10]:
    print(item)

Наиболее часто встречающиеся словосочетания длины 3 по нелемматизированному тексту:
нужно строительство можно
может остановиться можно
нужно только вспомнить
только можно надеяться
может уменьшиться только
когда нужно напомнить
только можно усвоить
будет нужно
нужно говорить ибо
нужно растерзание духа
